####ingesting races csv file

In [0]:
%sql
SHOW EXTERNAL LOCATIONS

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType
from pyspark.sql.functions import col, current_timestamp, to_timestamp, lit, concat

In [0]:
 races_schema = StructType([
    StructField("raceId", IntegerType(), False),
    StructField("year", IntegerType(), True),
    StructField("round", IntegerType(), True),
    StructField("circuitId", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("date", DateType(), True),
    StructField("time", StringType(), True),
    StructField("url", StringType(), True)
 ])

In [0]:
races_df = spark.read.csv("abfss://bronze@motorracingdl.dfs.core.windows.net/races.csv", header=True,           
                            schema=races_schema)

##### adding column ingestion and racetimestamp date to the dataframe

In [0]:
races_with_timestamp_df = races_df.withColumn("ingestion_date", current_timestamp())\
                                                .withColumn("race_timestamp", to_timestamp(concat(col("date"), lit(" "), col("time")), "yyyy-MM-dd HH:mm:ss"))
                                                

#####selecting required columns

In [0]:
races_selected_df = races_with_timestamp_df.select(col("raceId").alias("race_id"), col("year").alias("race_year"), col("round"), 
                                                   col("circuitId").alias("circuit_id"), col("name"), col("ingestion_date"), col("race_timestamp"))


##### write the data to the silver layer as parquet

In [0]:
#races_selected_df.write.mode("overwrite").partitionBy("race_year").parquet("abfss://silver@motorracingdl.dfs.core.windows.net/")